# Lightweight Face Detector TF Implementation

Based almost entirely off the paper: "A lightweight face detector by integrating the convolutional neural network with the image pyramid"

Github link: https://github.com/JiapengLuo/lightweight_face_detector


In [61]:
import tensorflow as tf
print(tf.__version__)

import os
import numpy as np
import time
from PIL import Image, ImageDraw

saved_model_path = "/Users/johngearig/Downloads/lightweight_face_detector-master/checkpoints/fast_mode_enable_hflip"
im_path = '/Users/johngearig/Downloads/lightweight_face_detector-master/demo1.jpg'
out_path = '/Users/johngearig/Downloads/lightweight_face_detector-master/demo1_detected.jpg'

1.13.1


NOTE: This will not run on any recent tensorflow release since it relies heavily on tf.Session() which was discontinued in tf.__version__ 2.0.

The Pillow import is only for viewing images and could be swapped for another image library. 

In [62]:
def draw_boxes(img, boxes, color='green', width=2):
    '''
    draw the boxes in the img
    :param img: Pillow Image or numpy
    :param boxes: boxes, [[ymax, xmax, ymin, xmin]...]
    :param color: color
    :return: Image drawed boxes
    '''
    if isinstance(img, np.ndarray):
        img = Image.fromarray(img.astype(np.uint8), mode='RGB')
    elif not isinstance(img, Image.Image):
        raise ValueError("image must be a Image or ndarray.")
    draw = ImageDraw.Draw(img)
    for box in boxes:
        draw.rectangle([box[1], box[0], box[3], box[2]], outline=color, width=width)
    return img

In [63]:
def run():
    sess = tf.compat.v1.Session()
    tf.saved_model.load(sess, ['serve'], saved_model_path )
    
    inp = sess.graph.get_tensor_by_name('input_images:0')
    boxes_tensor = sess.graph.get_tensor_by_name('boxes:0')
    scores_tensor = sess.graph.get_tensor_by_name('scores:0')

    im = Image.open(im_path)
    im = np.array(im).astype(np.float32)
    
    start = time.time()
    boxes, scores = sess.run([boxes_tensor,scores_tensor], feed_dict={inp: np.expand_dims(im, axis=0)})
    end = time.time()
    print(end - start)
    
    """print(boxes.shape)
    print(boxes.dtype)
    print(scores.shape)
    scores = np.reshape(scores, (1,200,1))
    
    universal = np.concatenate((boxes, scores))
    print(universal.shape)"""
    
    boxes = boxes[0]
    scores = scores[0]
    mask = scores>0.5
    boxes = boxes[mask]
    scores = scores[mask]
    print('boxes')
    print(boxes)
    print('scores')
    print(scores)
    out_im = draw_boxes(im, boxes)
    out_im.save(out_path)

if __name__ =='__main__':
    run()

INFO:tensorflow:Restoring parameters from /Users/johngearig/Downloads/lightweight_face_detector-master/checkpoints/fast_mode_enable_hflip/variables/variables
5.30732798576355
boxes
[[ 5.14040344e+02  1.17098621e+03  5.59904785e+02  1.20206531e+03]
 [ 1.48524887e+02  1.05480615e+03  1.85420395e+02  1.08539185e+03]
 [ 6.23874283e+01  1.38427271e+03  9.95855713e+01  1.40892139e+03]
 [ 8.89280090e+01  3.66035889e+02  1.27711845e+02  3.95632080e+02]
 [ 4.34563675e+01  6.84525696e+02  8.46375809e+01  7.13924988e+02]
 [ 1.31650696e+02  1.32913770e+03  1.68033646e+02  1.35700720e+03]
 [ 1.27111198e+02  6.71642212e+02  1.59609863e+02  6.96289307e+02]
 [ 2.81271729e+02  1.16416284e+03  3.17971100e+02  1.19078369e+03]
 [ 3.08307159e+02  3.95906067e+02  3.49407501e+02  4.28157715e+02]
 [ 2.37625885e+02  1.48838184e+03  2.80096710e+02  1.51913037e+03]
 [ 1.85299988e+02  4.91636200e+01  2.25629364e+02  8.42398529e+01]
 [ 9.47595825e+01  1.21035803e+03  1.32078613e+02  1.24074133e+03]
 [ 5.18268616e+

In [64]:
im = Image.open(out_path)  
im.show()




All code below is trying to figure out the correct reshapes needed to make compatible with other codebase




In [60]:

x = 5

boxes = np.random.rand(1,x,4)
scores = np.random.rand(1,x)

print(boxes.shape)
print(boxes.dtype)
print(scores.shape)

scores = np.reshape(scores, (1,x,1))
print(scores.shape)
universal = np.concatenate((boxes, scores),2)
print(universal.shape)

universal = np.reshape(universal, (1,1,x,5))
empty = np.zeros((1,1,x,2))
print(universal.shape)

universal = np.concatenate((empty, universal),3)
print(universal.shape)

universal

(1, 5, 4)
float64
(1, 5)
(1, 5, 1)
(1, 5, 5)
(1, 1, 5, 5)
(1, 1, 5, 7)


array([[[[0.        , 0.        , 0.51524304, 0.84366213, 0.78187815,
          0.34096203, 0.71766679],
         [0.        , 0.        , 0.07830278, 0.30447186, 0.46367956,
          0.76917488, 0.14501033],
         [0.        , 0.        , 0.12113918, 0.79679077, 0.5657318 ,
          0.49153919, 0.95983541],
         [0.        , 0.        , 0.89157484, 0.11556659, 0.53469272,
          0.9486069 , 0.69621516],
         [0.        , 0.        , 0.11779409, 0.14249121, 0.16024777,
          0.26372982, 0.11557692]]]])